In [ ]:
# need this code1q snippet in each notebook to mount the google 
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# install necessary packages
!pip install datasets==1.2.1
!pip install transformers==4.2.0
!pip install rouge_score
!pip install sentencepiece
!pip install bert-extractive-summarizer
!pip install summa

     |████████████████████████████████| 159 kB 8.4 MB/s 
     |████████████████████████████████| 243 kB 37.3 MB/s 
     |████████████████████████████████| 1.8 MB 9.3 MB/s 
     |████████████████████████████████| 895 kB 54.4 MB/s 
     |████████████████████████████████| 2.9 MB 49.3 MB/s 
     |████████████████████████████████| 1.2 MB 7.4 MB/s 
     |████████████████████████████████| 54 kB 2.4 MB/s 
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54411 sha256=dd1a320684e7fdfb4eedc06cbb19549e5ab25f960e64d9032b25c3a5ea051194
  Stored in directory: /root/.cache/pip/wheels/29/64/ac/7b443477588d365ef37ada30d456bdf5f07dc5be9f6324cb6e
Successfully built summa


In [ ]:
# import huggingface dataset class
from datasets import load_dataset, load_metric, Dataset

# Importing model and tokenizer
from transformers import PegasusForConditionalGeneration, Trainer, TrainingArguments, PegasusTokenizerFast

#Import extractive tools 
from summarizer import Summarizer
from summa.textrank import textrank

# Other packages
import pandas as pd
import numpy as np
import torch
import time

In [ ]:
# load dataset from pubmed:
train = load_dataset("scientific_papers", "pubmed", split="train")
val = load_dataset("scientific_papers", "pubmed", split="validation")
metric = load_metric("rouge")

#summarizer
summary_model = Summarizer()

Dataset scientific_papers downloaded and prepared to /root/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/043e40ed208b8a66ee9e8228c86874946c99d2fc6155a1daee685795851cfdfc. Subsequent calls will reuse this data.


Reusing dataset scientific_papers (/root/.cache/huggingface/datasets/scientific_papers/pubmed/1.1.1/043e40ed208b8a66ee9e8228c86874946c99d2fc6155a1daee685795851cfdfc)


In [ ]:
#Helper functions
def run_pegasus(chunk, model, tokenizer, device):
  batch = tokenizer(chunk, truncation=True, padding='longest', return_tensors="pt").to(device)
  pegasus_summary = tokenizer.batch_decode(model.generate(**batch), skip_special_tokens=True)
  return pegasus_summary[0]
  
def score(abstract, summary, article_count, number_of_articles):
    metric.add(prediction=summary, reference=abstract)
    #if (article_count + 1) % 10 == 0:
    #  print(f"{article_count + 1}/{number_of_articles} has been scored")

def run_validation(articles, model, tokenizer, device, title, abstract_length=None):
  counter = 0
  avg_arr = []
  start_time = time.time()
  while counter < len(articles['article']):
    abstract = articles['abstract'][counter]
    abstract_arr = abstract.split()
    avg_arr.append(len(articles['article'][counter].split()))
    if abstract_length:
      if len(abstract_arr) > abstract_length:
        abstract = ' '.join(abstract_arr[:abstract_length])
    peg = run_pegasus(articles['article'][counter], model, tokenizer, pegasus_device)
    score(abstract, peg, counter, len(articles['article']))
    counter += 1
  end_time = time.time()
  print("Runtime: " + str(end_time - start_time))
  df = pd.DataFrame({title : avg_arr})
  print(df.describe())
  output = metric.compute(rouge_types=['rouge1', 'rouge2', 'rouge3', 'rougeL'])
  low = []
  mid = []
  high = []
  for rt in output:
    low.append(output[rt].low.fmeasure)
    mid.append(output[rt].mid.fmeasure)
    high.append(output[rt].high.fmeasure)
  print(pd.DataFrame({"Low" : low, "Mid" : mid, "High" : high}, index=["Rouge1", "Rouge2", "Rouge3", "RougeL"]))

def run_summarier(num_articles, arr, tp):
  article_arr = []
  counter = 0
  while counter < num_articles:
    article = arr[counter]["article"]
    article_len = len(article.split())
    if article_len > 512:
      if tp == "BERT":
        result = summary_model(arr["article"][counter], ratio=(512/article_len))  # Specified with ratio
      elif tp == "TextRank":
        result = textrank(arr["article"][counter], ratio=(512/article_len))  # Specified with ratio
      else:
        print("Summariation type not found")
        break
    else:
      result = article
    article_arr.append({"abstract": arr[counter]["abstract"], "article":result})
    if (counter + 1) % 10 == 0:
      print(f"{counter + 1}/{num_articles} has been summarized")
    counter += 1
  return Dataset.from_pandas(pd.DataFrame(article_arr))

In [ ]:
#training
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        #item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)

      
def prepare_data(model_name, train_texts, train_labels, val_texts, val_labels):
  tokenizer = PegasusTokenizerFast.from_pretrained(model_name)
  prepare_val = False if val_texts is None or val_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt", max_length=512)
    decodings = tokenizer(labels, truncation=True, padding=True, return_tensors="pt", max_length=512)
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None

  return train_dataset, val_dataset, tokenizer


def prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset, epochs, train_batch, val_batch):
  torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
  model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

  # Establish hyperparameters
  model.config.num_beams = 8
  model.config.max_length = 512
  model.config.length_penalty = 0.8 #beam alpha

  training_args = TrainingArguments(
    output_dir='/content/gdrive/MyDrive/w266_project/PEGASUS_Tuning',           # output directory
    overwrite_output_dir=True,
    num_train_epochs=epochs,              # total number of training epochs
    per_device_train_batch_size=train_batch,   # batch size per device during training, can increase if memory allows
    per_device_eval_batch_size=val_batch,    # batch size for evaluation, can increase if memory allows
    save_steps=2000,                  # number of updates steps before checkpoint saves
    save_total_limit=1,              # limit the total amount of checkpoints and deletes the older checkpoints
    evaluation_strategy='steps',     # evaluation strategy to adopt during training
    eval_steps=1000,                 # number of update steps before evaluation
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
    #Added
    learning_rate=5e-4,
    #fp16=True,                      #Whether to use 16-bit (mixed) precision training instead of 32-bit training.
    label_smoothing_factor=0.1,
    adafactor=True,
  )

  trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    tokenizer=tokenizer
  )

  return trainer

In [ ]:
#Training
train_texts, train_labels = train['article'][:100], train['abstract'][:100]
val_texts, val_labels = val['article'][:10], val['abstract'][:10]
model_name = 'google/pegasus-large'

train_dataset, val_dataset, tokenizer = prepare_data(model_name, train_texts, train_labels, val_texts, val_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset, epochs=20, train_batch=1, val_batch=1)
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:557: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1025.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,2.312300,3.607160,1.230500,8.127000
2000,1.616000,3.696308,1.230400,8.128000


TrainOutput(global_step=2000, training_loss=1.9641088256835937, metrics={'train_runtime': 1232.9324, 'train_samples_per_second': 1.622, 'total_flos': 3506977112064000, 'epoch': 20.0})

In [ ]:
#Train PEGASUS with BERT
count = 100

bert_train = run_summarier(count, train, "BERT")
train_texts, train_labels = bert_train['article'][:100], bert_train['abstract'][:100]
val_texts, val_labels = val['article'][:10], val['abstract'][:10]
model_name = 'google/pegasus-large'

train_dataset, val_dataset, tokenizer = prepare_data(model_name, train_texts, train_labels, val_texts, val_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset, epochs=20, train_batch=1, val_batch=1)
trainer.train()

10/100 has been summarized
20/100 has been summarized
30/100 has been summarized
40/100 has been summarized
50/100 has been summarized
60/100 has been summarized
70/100 has been summarized
80/100 has been summarized
90/100 has been summarized
100/100 has been summarized


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,2.336300,3.648534,1.236500,8.088000
2000,1.616200,3.720283,1.234100,8.103000


TrainOutput(global_step=2000, training_loss=1.9762465209960938, metrics={'train_runtime': 1233.4715, 'train_samples_per_second': 1.621, 'total_flos': 3506977112064000, 'epoch': 20.0})

In [ ]:
#Train PEGASUS with TextRank
count = 100

textrank_train = run_summarier(count, train, 'TextRank')
train_texts, train_labels = textrank_train['article'][:100], textrank_train['abstract'][:100]
val_texts, val_labels = val['article'][:10], val['abstract'][:10]
model_name = 'google/pegasus-large'

train_dataset, val_dataset, tokenizer = prepare_data(model_name, train_texts, train_labels, val_texts, val_labels)
trainer = prepare_fine_tuning(model_name, tokenizer, train_dataset, val_dataset, epochs=20, train_batch=1, val_batch=1)
trainer.train()

10/100 has been summarized
20/100 has been summarized
30/100 has been summarized
40/100 has been summarized
50/100 has been summarized
60/100 has been summarized
70/100 has been summarized
80/100 has been summarized
90/100 has been summarized
100/100 has been summarized


Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,2.344000,3.672385,1.230500,8.127000
2000,1.614400,3.726747,1.235100,8.096000


TrainOutput(global_step=2000, training_loss=1.9792006225585939, metrics={'train_runtime': 1237.0711, 'train_samples_per_second': 1.617, 'total_flos': 3506977112064000, 'epoch': 20.0})

In [ ]:
#Get Results
pegasus_device = 'cuda' if torch.cuda.is_available() else 'cpu'
pegasus_tokenizer = PegasusTokenizerFast.from_pretrained('google/pegasus-large')
#PEGASUS BASE
pegasus_base = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(pegasus_device)
#PEGASUS Train 500
pegasus_train_20_dir = "/content/gdrive/MyDrive/w266_project/PEGASUS_20/"
pegasus_train_20 = PegasusForConditionalGeneration.from_pretrained(pegasus_train_20_dir).to(pegasus_device)
#PEGASUS Train BERT
pegasus_train_BERT_dir = "/content/gdrive/MyDrive/w266_project/PEGASUS_BERT/"
pegasus_train_BERT = PegasusForConditionalGeneration.from_pretrained(pegasus_train_BERT_dir).to(pegasus_device)
#PEGASUS TextRank
pegasus_train_TextRank_dir = "/content/gdrive/MyDrive/w266_project/PEGASUS_TextRank/"
pegasus_train_TextRank = PegasusForConditionalGeneration.from_pretrained(pegasus_train_TextRank_dir).to(pegasus_device)

count = 100
#skip the first 10 examples since we used those for validation in training
test = Dataset.from_dict(val[11:])

bert_test = run_summarier(count, test, 'BERT')
textrank_test = run_summarier(count, test, 'TextRank')

10/100 has been summarized
20/100 has been summarized
30/100 has been summarized
40/100 has been summarized
50/100 has been summarized
60/100 has been summarized
70/100 has been summarized
80/100 has been summarized
90/100 has been summarized
100/100 has been summarized
10/100 has been summarized
20/100 has been summarized
30/100 has been summarized
40/100 has been summarized
50/100 has been summarized
60/100 has been summarized
70/100 has been summarized
80/100 has been summarized
90/100 has been summarized
100/100 has been summarized


In [ ]:
#Original and Trained
run_validation(articles=test[:count], model=pegasus_base, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS BASE")
run_validation(articles=test[:count], model=pegasus_train_20, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS with training")
#BERT
run_validation(articles=test[:count], model=pegasus_train_BERT, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS BERT")
run_validation(articles=bert_test[:count], model=pegasus_train_BERT, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS BERT with bert_val")
#TextRank
run_validation(articles=test[:count], model=pegasus_train_TextRank, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS TextRank")
run_validation(articles=textrank_test[:count], model=pegasus_train_TextRank, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS TextRank with textrank_val")
#Mixed Models
run_validation(articles=textrank_test[:count], model=pegasus_train_BERT, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS BERT with textrank_val")
run_validation(articles=bert_test[:count], model=pegasus_train_TextRank, tokenizer=pegasus_tokenizer, device=pegasus_device, title="PEGASUS TextRank with bert_val")

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Runtime: 424.1110248565674
       PEGASUS BASE
count    100.000000
mean    2793.360000
std     1618.468565
min      438.000000
25%     1465.750000
50%     2402.000000
75%     3838.000000
max     8904.000000
             Low       Mid      High
Rouge1  0.243544  0.262696  0.284260
Rouge2  0.067279  0.083432  0.101420
Rouge3  0.031929  0.045058  0.058661
RougeL  0.151118  0.165167  0.181291
Runtime: 686.5192625522614
       PEGASUS with training
count             100.000000
mean             2793.360000
std              1618.468565
min               438.000000
25%              1465.750000
50%              2402.000000
75%              3838.000000
max              8904.000000
             Low       Mid      High
Rouge1  0.314631  0.339341  0.364674
Rouge2  0.097661  0.121639  0.154583
Rouge3  0.046043  0.071120  0.100843
RougeL  0.191592  0.212195  0.239823
Runtime: 711.5255360603333
       PEGASUS BERT
count    100.000000
mean    2793.360000
std     1618.468565
min      438.000000
25%     